In [ ]:
from __future__ import division
import numpy as np
from scipy.integrate import solve_ivp
from scipy import interpolate 
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from datetime import date
import datetime
from myfunctions_multi_scale import *
%matplotlib inline

In [ ]:
# Sensitivity analysis functions
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol
import seaborn as sns

# sensitivity analysis is based on data from 2014 

In [ ]:
# Import of I data
# irradiance data is based on meteorological data from: https://ims.data.gov.il/he/ims/6 
dFI = pd.read_csv('../data/ims_data_2014_umol_photons.csv',encoding= 'unicode_escape',header=None)
days = list(range(1,366))

In [ ]:
I_plot = np.zeros(24*365)
light_hours = list(range(5,19)) #light hours defined according to longest days 
for day in days:
    for hour in light_hours:
        I_plot[(day - 1)*24 + hour] = float(dFI.iloc[day][hour-1])

annual_hours = list(range(1,(366-1)*24+1))
f0 = interpolate.interp1d(annual_hours, I_plot,kind = 'linear')

In [ ]:
# T and light data

x1 = pd.ExcelFile('../data/input.xlsx')

dfT = x1.parse('T_multi-scale',header=None)
# Temperatures are water temperatures, taken from:
# Y. Suari, et al., Sandbar Breaches Control of the Biogeochemistry of a Micro-Estuary RIME-restoration of
# Israeli micro estuaries. Front. Mar. Sci. (2019) 

days_reduced,hours_reduced = [],[]
for i in range(1,13):
    day = datetime.datetime(2014, i, 1)
    days_reduced.append(int(day.strftime("%j")))

days_reduced.append(366)
for j in range(1,13):
    day = datetime.datetime(2014, j, 1)
    hours_reduced.append((int(day.strftime("%j"))-1)*24)
hours_reduced.append((366-1)*24)
T = []

# assigmment of parameters to values:
for key,val in zip(dfT.iloc[1:][0],dfT.iloc[1:][1]):
    exec(key + '=val')
    T.append(val)
    print(key,val)
T.append(T[0])
days = list(range(1,366))


# Interpolate T data (T_interp)

f1 = interpolate.interp1d(hours_reduced, T,kind = 'cubic')

T_interp = f1(annual_hours)


fig, ax = plt.subplots(2,1,figsize=(16,10))
xlabels = ['1 Jan', '1 February', '1 March', '1 April', '1 May', '1 June', '1 July', '1 August', '1 September', '1 October','1 November', '1 December']

ax[0].plot(annual_hours,T_interp, 'r-')
ax[1].plot(annual_hours,I_plot, 'b-', linewidth=0.5)

ax[1].set_xlabel('Time',fontsize=14, weight="bold")
ax[0].set_ylabel('Temperature \n[C]',fontsize=10, weight="bold")
ax[1].set_ylabel('Irradiance \n[umol photons / m2 / second]',fontsize=10, weight="bold")
ax[0].set_xticklabels([])
ax[1].set_xticklabels([])
ax[1].set_xticks(annual_hours[0:len(annual_hours):int(len(annual_hours)/12-1)])
ax[1].set_xticklabels([str(i) for i in xlabels], rotation=45,fontsize=10, weight="bold")

In [ ]:
I = np.zeros(24*365)
light_hours = list(range(5,19))
for day in days:
    for hour in light_hours:
        I[(day - 1)*24 + hour] = 0.43 * float(dFI.iloc[day][hour-1]) # 0.43 is PAR constant

annual_hours = list(range(1,(366-1)*24+1))
f0 = interpolate.interp1d(annual_hours, I,kind = 'linear')

In [ ]:
df1 = x1.parse('Parameters_multi-scale',header=None)

# assigmment of parameters to values: 
for key,val in zip(df1.iloc[:][0],df1.iloc[:][1]):
    exec(key + '=val')
    print(key,val)

n_reactors = np.int(n_reactors)

In [ ]:
# sensitivity analysis definition: examined parameters and value ranges

problem = {
    'num_vars': 20,
    'names': ['Qp','µmax', 'Nintcrit','Nintmax','Nintmin','Ks','Vmax','KI','K0','Ka', 'n','Tmax','Topt','Tmin','λ20','Sopt','Smax','Smin','teta','dilution'],
    'bounds': [[300,600],
               [0.025, 0.035],
               [1.5, 3],
               [3.2,4.5],
               [0.5,0.7],
               [10,30],
               [50,250],
               [15, 150],
               [0.1,3],
               [0.01,0.2],
               [1, 6],
               [29, 32],
               [15, 20],
               [1,10],
               [0.001,0.005],
               [15,25],
               [40,50],
               [0,10],
               [0.9,1.2],
               [0,0.05]]
}
print(problem)

In [ ]:
n_reactors = 100

In [ ]:
# number of examined values per parameter - 10
param_values = saltelli.sample(problem, 10)
print(param_values.shape)

In [ ]:
print(param_values)

In [ ]:
# 4 seasons - one cultivation period (14 days) per season

Y1 = np.zeros([param_values.shape[0]])
Y2 = np.zeros([param_values.shape[0]])
Y3 = np.zeros([param_values.shape[0]])
evaluate_model1 = []
evaluate_model2 = []
evaluate_model3 = []
x0 = n_reactors*[Nsea, Next0, Nint0, m0]
resolution = 1 # 1 out of how many cages is presented?
dilution = 0
weeks = [1, 9, 32, 46]
seasons = ['Winter', 'Spring', 'Summer','Automn']
k = 0
for j, X in enumerate(param_values):
    Qp = X[0]
    miu = X[1]
    Nintcrit = X[2]
    Nintmax = X[3]
    Nintmin = X[4]
    Ks = X[5]
    Vmax = X[6]
    KI = X[7]
    K0 = X[8]
    Ka = X[9]
    n = X[10]
    Tmax = X[11]
    Topt = X[12]
    Tmin = X[13]
    losses20 = X[14]
    Sopt = X[15]
    Smax = X[16]
    Smin = X[17]
    teta = X[18]
    dilution = X[19]
    
    Total_biomass = []
    Total_N = []
    NSEA, NEXT, NINT, M, Total_N, T = [],[],[],[],[],[]
    NSEA_F = []
        
    for week0 in weeks:
        t0 = 10 + 7 * 24 * (week0-1)
        t = list(range(t0,t0 + int(n_days*24)))
        times = list(range(t0,t0 + int(n_days*24)))
        args = (Nintcrit,Nintmax,Nintmin,Vmax,Ks,KN,miu,S,Z,KI,K0,Ka,Topt,Tmin,Tmax,losses20,
                teta,Sopt,Smin,Smax, Qp, Qsea, Nsea,f1,f0,dilution,n,umol_to_percent_DW)

        # solve the ODEs using the new syntax
        sol = solve_ivp(multi_N_f_un, [t[0], t[-1]], x0, args=args, t_eval = t)

        # take the solution of the state variables:
        M_farm, N_farm = [],[]
        for i in range(n_reactors):
            NSEA.append(sol.y[i*4,:])
            NEXT.append(sol.y[i*4+1,:])
            NINT.append(sol.y[i*4+2,:])
            M.append(sol.y[i*4+3,:])
            M_farm.append((M[-1][-1] - m0) * 1.785) # 1.785 is cage volume
            N_farm.append((M[-1][-1] * NINT[-1][-1] * 1.785 / 100)) # unit conversion to N removal
        Total_biomass.append(round(sum(M_farm),3))
        Total_N.append(round(sum(N_farm),4))
        NSEA_F.append(round(NSEA[-1][-1],3))
        T.append(sol.t)

        NSEA, NEXT, NINT, M, N_farm, T = [],[],[],[],[],[]
        x0 = n_reactors*[Nsea, Next0, Nint0, m0]
    evaluate_model1.append(np.sum(Total_biomass))
    evaluate_model2.append(np.sum(Total_N))
    evaluate_model3.append(np.mean(NSEA_F))                       
    Y1[j] = round(evaluate_model1[-1],4)
    Y2[j] = round(evaluate_model2[-1],4)
    Y3[j] = round(evaluate_model3[-1],4)
    print(k)
    k = k+1

In [ ]:
print(min(Y1))
print(np.mean(Y1))
print(np.std(Y1))
print(np.median(Y1))
print(max(Y1))

In [ ]:
Si1 = sobol.analyze(problem, Y1,print_to_console=True)

In [ ]:
print(min(Y2))
print(np.mean(Y2))
print(np.std(Y2))
print(np.median(Y2))
print(max(Y2))

In [ ]:
Si2 = sobol.analyze(problem, Y2,print_to_console=True)

In [ ]:
print(min(Y3))
print(np.mean(Y3))
print(np.std(Y3))
print(np.median(Y3))
print(max(Y3))

In [ ]:
Si3 = sobol.analyze(problem, Y3,print_to_console=True)

In [ ]:
fig,ST2 = plt.subplots(1,1,sharex=True,figsize=(4,5))
ST2.plot(Si3['ST'],problem['names'],'s',markersize=5,color='dimgray')
ST2.plot(Si2['ST'],problem['names'],'*',markersize=7,color='dodgerblue')
ST2.plot(Si1['ST'],problem['names'],'.',markersize=10,color='black')


ST2.set_xlabel('Sensitivity Index',fontsize=14, weight="bold")
ST2.set_ylabel('Parameter',fontsize=14, weight="bold")
ST2.set_xlim([0, 1])
ST2.set_axisbelow(True)
ST2.yaxis.grid(color='lightgray', linestyle='dashed')
ST2.legend(['Environmental N concentration','N sequestration','Biomass production'])


name = 'fig3.png' 
fig.savefig(fname=name, dpi=600)